In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats


df = pd.read_csv('../data/benin-malanville.csv')

print(df.describe())

#Missing value

missing_report = df.isna().sum() / len(df) * 100
print("Missing values (%):\n", missing_report[missing_report > 5])


# Z-score

numeric_cols = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']
z_scores = np.abs(stats.zscore(df[numeric_cols].dropna()))
outliers = (z_scores > 3).any(axis=1)
print(f"Found {outliers.sum()} outliers (|Z| > 3).")

# Drop outliers
df_clean = df[~outliers].copy()

#Impute with median for missing values
for col in numeric_cols:
    df_clean[col].fillna(df_clean[col].median(), inplace=True)

ModuleNotFoundError: No module named 'pandas'